
# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# Chapitre 9 : Classification bayésienne naïve

### Description

La théorie de ce chapitre est illustrée pour classer les messages électroniques en courriels et pourriels.

### Données

Les données exploitées dans ce chapitre correspondents aux messages électroniques authentiques d'un employé de la compagnie Enron reçus entre le 28 décembre 2003 et le 6 septembre 2005. Vous pouvez télécharger le jeux de données (*Enron1.zip*) compressé à partir du site web du cours. Le fichier doit être décompressé dans le répertoire courant courant de votre calepin Jupyter. 

Notez que les messages électroniques de 158 employés de la compagnie Enron ont été récupérés par la Federal Energy Regulatory Commission pendant la commission d'enquête qui a eu lieu après l'effondrement de la compagnie. Dans ce chapitre, nous n'utilisons que les messages d'un seul employé. Vous pouvez récupérer beaucoup plus de données sur Enron entier à suivante  https://www.cs.cmu.edu/~enron/.

# Chargement des librairies

In [ ]:
using DataFrames, Distributions, Gadfly, MLBase, Random
Random.seed!(3302);

In [ ]:
# using Cairo, Fontconfig

# Chargement des données

Le code de cette section permet de traiter les fichiers textes correspondant à tous les messages électroniques de l'utilisateurs. Les messages électroniques se trouvent dans les dossier *ham/* et *spam/* de l'utilisateur *Enron1*.

In [ ]:
# Récupération des noms de fichier de tous les hams
filesdir = "enron1/ham/"
filename_ham = filesdir.*readdir(filesdir);

In [ ]:
# Récupération des noms de fichier de tous les spams
filesdir = "enron1/spam/"
filename_spam = filesdir.*readdir(filesdir);

### Partitionnement des données en ensemble d'entraînement et de validation

Le 2/3 des données constituent l'ensemble d'entraînement et le 1/3 restant l'ensemble de validation.

In [ ]:
# Partitionnement des courriels
ham_train = sample(filename_ham, round(Int, 2/3*length(filename_ham)), replace=false, ordered=true)
ham_valid = setdiff(filename_ham, ham_train);

In [ ]:
# Partitionnement des pourriels
spam_train = sample(filename_spam, round(Int, 2/3*length(filename_spam)), replace=false, ordered=true)
spam_valid = setdiff(filename_spam, spam_train);

In [ ]:
# Vecteur des solutions de l'ensemble de validation (0 = pourriel, 1 = courriel)
Z = vcat(ones(Int64, length(ham_valid)), zeros(Int64, length(spam_valid)));

In [ ]:
println("L'échantillon d'entraînement est composé de $(length(ham_train)) courriels et $(length(spam_train)) pourriels.")
println("L'échantillon de validation est composé de $(length(ham_valid)) courriels et $(length(spam_valid)) pourriels.")

# Extraction des occurrences des mots

### Fonctions permettants le traitement des fichiers textes pour la classifications bayésienne naïve.

In [ ]:
"""
    wordlisting(filename::String)

Extrait la liste des mots contenus dans le fichier texte `filename`.

### Détails
Ne dénombre pas le nombre d'occurrence des mots dans un fichier. N'est pas sensible aux majuscules ni aux minuscules.
"""
function wordlisting(filename::String)
    
    f = read(filename, String)
    text = replace(f, r"[0123456789]" => "")
    words = split(text, r"\W+")
    filter!(x -> length(x) > 1, words)
    wordlist = unique(words)
    
end

"""
    wordcounting(wordlist::Vector{<:AbstractString})

Dénombre les occurrences des mots dans la liste `wordlist`. 

### Détails
Retourne un dictionnaire ayant comme clé le mot, et la valeur l'occurrence du mot.
"""
function wordcounting(wordlist::Vector{<:AbstractString})

    wordcounts = Dict{String,Int64}()

    for word in wordlist
        wordcounts[word]=get(wordcounts, word, 0) + 1
    end
    
    return wordcounts
    
end

### Extraction de l'occurrence des mots des courriels

In [ ]:
ham_wordlist = wordlisting.(ham_train)
ham_wordcounts = wordcounting(vcat(ham_wordlist...))

### Extraction de l'occurrence des mots des courriels

In [ ]:
spam_wordlist = wordlisting.(spam_train)
spam_wordcounts = wordcounting(vcat(spam_wordlist...))

## 9.1 Modèle marginal

### Loi *a priori*

In [ ]:
α, β = (1,1)

### Dénombrement des courriels et des pourriels de l'ensemble d'entraînement

In [ ]:
# nombre de courriels dans l'ensemble d'entraînement
n₀ = length(spam_train)

# nombre de pourriels dans l'ensemble d'entraînement
n₁ = length(ham_train)

# taille de l'échantillon d'entraînement
n = n₀ + n₁

### Loi *a posteriori*

In [ ]:
fd = Beta(α+n₁, β+n₀)
fig = plot(Guide.xlabel("θ"), Guide.ylabel("f(θ)"), Guide.colorkey(title=""),
        layer(x -> pdf(Beta(α, β),x), 0, 1, color=["loi a priori"]),
        layer(x -> pdf(fd, x), .0, 1, color=["loi a posteriori"])
        )
# draw(PDF("posterior.pdf"), fig)

### Probabilité prédictive

In [ ]:
# Probabilité que le nouveau message soit un courriel 
p₁ = (α + n₁)/(α + β + n)

In [ ]:
# Probabilité que le nouveau message soit un pourriel 
p₀ = (β + n₀)/(α + β + n)

### Prédiction sur l'ensemble de validation

In [ ]:
# Tous les nouveaux messages sont classés comme courriels avec le modèle marginal
Ẑ = ones(Int64, length(Z))

r = roc(Z, Ẑ)

In [ ]:
println("Sur les $(r.p+r.n) messages de l'ensemble de test,")
println("")
println("- on classe $(r.tp) courriels comme courriels ;")
println("- on classe $(r.tn) pourriels comme pourriels ;")
println("")
println("- on classe $(r.fp) pourriels comme courriels ;")
println("- on classe $(r.fn) courriels comme pourriels ;")
println("")
println("- le score F₁ du classement est de ", round(f1score(r),digits=2), ".")

## 9.2 Modèle à une variable explicative

### Loi *a priori*

In [ ]:
α₀₁, β₀₁ = (1,1)
α₁₁, β₁₁ = (1,1)

### Dénombrement des courriels et des pourriels contenant le mot *http*

In [ ]:
word1 = "http"

n₀₁ = spam_wordcounts[word1]
n₁₁ = ham_wordcounts[word1]

println("Le mot ", word1, " est présent dans ", n₀₁, " pourriels et ", n₁₁, " courriels.")


### Probabilités prédictives

#### (i) La probabilité que le message soit un courriel sachant qu'il contient le mot *http*.

In [ ]:
p₀ = (β + n₀)/(α + β + n) * (α₀₁ + n₀₁)/(α₀₁ + β₀₁ + n₀)
p₁ = (α + n₁)/(α + β + n ) * (α₁₁ + n₁₁)/(α₁₁ + β₁₁ +n₁)

# constante de normalisation
c = p₀ + p₁
p₀ = p₀/c
p₁ = p₁/c

println("Si le mot http est présent dans le message, il y a une prob de $p₁ que ce soit un courriel.")

#### (ii) La probabilité que le message soit un courriel sachant qu'il ne contient pas le mot *http*.

In [ ]:
p₀ = (β + n₀)/(α + β + n) * (β₀₁ + n₀ - n₀₁)/(α₀₁ + β₀₁ + n₀)
p₁ = (α + n₁)/(α + β + n ) * (β₁₁ + n₁ - n₁₁)/(α₁₁ + β₁₁ + n₁)

# constante de normalisation
c = p₀ + p₁
p₀ = p₀/c
p₁ = p₁/c

println("Si le mot http n'est pas présent dans le message, il y a une prob de $p₁ que ce soit un courriel.")

### Prédiction sur l'ensemble de validation

In [ ]:
Ẑ = Int64[]

for filename in ham_valid
      
    wordlist = wordlisting(filename)
    x̃ = any(wordlist .== word1)
    
    if x̃
        push!(Ẑ, 0)
    else
        push!(Ẑ, 1)
    end
 
end

for filename in spam_valid
      
    wordlist = wordlisting(filename)
    x̃ = any(wordlist .== word1)
    
    if x̃
        push!(Ẑ, 0)
    else
        push!(Ẑ, 1)
    end
 
end

r = roc(Z, Ẑ)

In [ ]:
println("Sur les $(r.p+r.n) messages de l'ensemble de test,")
println("")
println("- on classe $(r.tp) courriels comme courriels ;")
println("- on classe $(r.tn) pourriels comme pourriels ;")
println("")
println("- on classe $(r.fp) pourriels comme courriels ;")
println("- on classe $(r.fn) courriels comme pourriels ;")
println("")
println("- le score F₁ du classement est de ", round(f1score(r),digits=2), ".")

## 9.3 Modèle à deux variables

### Loi *a priori*

In [ ]:
α₀₂, β₀₂ = (1,1)
α₁₂, β₁₂ = (1,1)

### Dénombrement des courriels et des pourriels contenant le mot *enron*

In [ ]:
word2 = "enron"

if haskey(spam_wordcounts, word2)
    n₀₂ = spam_wordcounts[word2]
else
    n₀₂ = 0
end

if haskey(ham_wordcounts, word2)
    n₁₂ = ham_wordcounts[word2]
else
    n₁₂ = 0
end


println("Le mot ", word2, " est présent dans ", n₀₂, " pourriels et ", n₁₂, " courriels.")

### Probabilités prédictives

#### (i) La probabilité que le message soit un courriel sachant qu'il contient les mot *http* et *enron*.

In [ ]:
p₀ = (β + n₀)/(α + β + n) * (α₀₁ + n₀₁)/(α₀₁ + β₀₁ + n₀) * (α₀₂ + n₀₂)/(α₀₂ + β₀₂ + n₀)
p₁ = (α + n₁)/(α + β + n) * (α₁₁ + n₁₁)/(α₁₁ + β₁₁ + n₁) * (α₁₂ + n₁₂)/(α₁₂ + β₁₂ + n₁)

# constante de normalisation
c = p₀ + p₁
p₀ = p₀/c
p₁ = p₁/c

println("Si les mots $word1 et $word2 sont présents dans le message, il y a une prob de $p₁ que ce soit un courriel.")

#### (ii) La probabilité que le message soit un courriel sachant qu'il contient le mot *http* mais ne contient pas le mot *enron*.

In [ ]:
p₀ = (β + n₀)/(α + β + n) * (α₀₁ + n₀₁)/(α₀₁ + β₀₁ + n₀) * (β₀₂ + n₀ - n₀₂)/(α₀₂ + β₀₂ + n₀)
p₁ = (α + n₁)/(α + β + n) * (α₁₁ + n₁₁)/(α₁₁ + β₁₁ + n₁) * (β₁₂ + n₁ - n₁₂)/(α₁₂ + β₁₂ + n₁)

# constante de normalisation
c = p₀ + p₁
p₀ = p₀/c
p₁ = p₁/c

println("Si le mot $word1 est présent mais $word2 est absent, il y a une prob de $p₁ que ce soit un courriel.")

#### (iii) La probabilité que le message soit un courriel sachant qu'il ne contient pas le mot *http* mais contient le mot *enron*.

In [ ]:
p₀ = (β + n₀)/(α + β + n) * (β₀₁ + n₀ - n₀₁)/(α₀₁ + β₀₁ + n₀) * (α₀₂ + n₀₂)/(α₀₂ + β₀₂ + n₀)
p₁ = (α + n₁)/(α + β + n) * (β₁₁ + n₁ - n₁₁)/(α₁₁ + β₁₁ + n₁) * (α₁₂ + n₁₂)/(α₁₂ + β₁₂ + n₁)

# constante de normalisation
c = p₀ + p₁
p₀ = p₀/c
p₁ = p₁/c

println("Si le mot $word1 est présent mais $word2 est absent, il y a une prob de $p₁ que ce soit un courriel.")

#### (iv) La probabilité que le message soit un courriel sachant qu'il ne contient pas les mot *http* et *enron*.

In [ ]:
p₀ = (β + n₀)/(α + β + n) * (β₀₁ + n₀ - n₀₁)/(α₀₁ + β₀₁ + n₀) * (β₀₂ + n₀ - n₀₂)/(α₀₂ + β₀₂ + n₀)
p₁ = (α + n₁)/(α + β + n) * (β₁₁ + n₁ - n₁₁)/(α₁₁ + β₁₁ + n₁) * (β₁₂ + n₁ - n₁₂)/(α₁₂ + β₁₂ + n₁)


# constante de normalisation
c = p₀ + p₁
p₀ = p₀/c
p₁ = p₁/c

println("Si les mots $word1 et $word2 ne sont pas présents dans le message, il y a une prob de $p₁ que ce soit un courriel.")

### Prédiction sur l'ensemble de validation

In [ ]:
Ẑ = Int64[]

for filename in ham_valid
      
    wordlist = wordlisting(filename)
    x̃₁ = any(wordlist .== word1)
    x̃₂ = any(wordlist .== word2)
    
    if x̃₁ && !x̃₂
        push!(Ẑ, 0)
    else
        push!(Ẑ, 1)
    end
 
end

for filename in spam_valid
      
    wordlist = wordlisting(filename)
    x̃₁ = any(wordlist .== word1)
    x̃₂ = any(wordlist .== word2)
    
    if x̃₁ && !x̃₂
        push!(Ẑ, 0)
    else
        push!(Ẑ, 1)
    end
 
end

r = roc(Z, Ẑ)

In [ ]:
println("Sur les $(r.p+r.n) messages de l'ensemble de test,")
println("")
println("- on classe $(r.tp) courriels comme courriels ;")
println("- on classe $(r.tn) pourriels comme pourriels ;")
println("")
println("- on classe $(r.fp) pourriels comme courriels ;")
println("- on classe $(r.fn) courriels comme pourriels ;")
println("")
println("- le score F₁ du classement est de ", round(f1score(r),digits=2), ".")